<a href="https://colab.research.google.com/github/d2021101420/MonoDepth-PyTorch/blob/master/Mauricio_Monodepth_200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# clone do meu repositorio editado de https://github.com/OniroAI/MonoDepth-PyTorch
!git clone https://github.com/d2021101420/MonoDepth-PyTorch

Cloning into 'MonoDepth-PyTorch'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 208 (delta 6), reused 0 (delta 0), pack-reused 193
Receiving objects: 100% (208/208), 34.60 MiB | 30.39 MiB/s, done.
Resolving deltas: 100% (112/112), done.


In [ ]:
%cd MonoDepth-PyTorch/

/content/MonoDepth-PyTorch


In [ ]:
#Apagar dataset original
#!rm kitti_archives_to_download.txt
#Carregar arquivo com indice de datasets
#from google.colab import files
#files.upload()

In [ ]:
!mkdir data
%cd data
!mkdir models
%cd models
!pwd

/content/MonoDepth-PyTorch/data
/content/MonoDepth-PyTorch/data/models
/content/MonoDepth-PyTorch/data/models


In [ ]:
#Carregar modelo do meu drive


#!wget https://p-def6.pcloud.com/cBZcIboSHZMPuC9HZZZDTMLv7Z2ZZfLRZkZkCcA0ZskZE5ZqXZqHZ10Z5kZHZI7ZoFZ8VZS7ZwVZb0ZYXZb5r97Z8VQ93u4jwMpRRuv6zpzmoHFV8cIV/monodepth_resnet18_001.pth -P ~/data/models

import gdown
url='https://drive.google.com//uc?id=1n6_RqYurD8-EqY121uSyWZUwWHQX-mPG'
output='monodepth_resnet18_001.pth'
#time.sleep(100)
gdown.download(url, output, quiet=False)
#!gdown --id https://drive.google.com/file/d/1n6_RqYurD8-EqY121uSyWZUwWHQX-mPG
#from google.colab import files
#files.upload()

Downloading...
From: https://drive.google.com//uc?id=1n6_RqYurD8-EqY121uSyWZUwWHQX-mPG
To: /content/MonoDepth-PyTorch/data/models/monodepth_resnet18_001.pth
81.0MB [00:00, 119MB/s]


'monodepth_resnet18_001.pth'

In [ ]:
#verificar carga do modelo

!ls -l 

total 79132
-rw-r--r-- 1 root root 81027095 Jun 21 19:11 monodepth_resnet18_001.pth


In [ ]:
#alterar para o diretorio do modelo
%cd /content/MonoDepth-PyTorch/

/content/MonoDepth-PyTorch


## Train

Check if CUDA is available

In [ ]:
import os
import torch
import numpy as np
import skimage.transform
import matplotlib.pyplot as plt
from easydict import EasyDict as edict

from main_monodepth_pytorch import Model
%reload_ext autoreload
%autoreload 2

In [ ]:
torch.cuda.is_available()

True

In [ ]:
torch.cuda.device_count()

1

In [ ]:
torch.cuda.empty_cache()

IMPORTAÇÃO DOS DATASETs de Treinamento, Validação e testes

Dataset escolhido = Mid-Air, The Montefiore Institute Dataset of Aerial Images and Records

https://midair.ulg.ac.be/

When using this dataset in your research, we would appreciate that you cite our CVPRW paper:

@INPROCEEDINGS{Fonder2019MidAir,
author = {Michael Fonder and Marc Van Droogenbroeck},
title = {Mid-Air: A multi-modal dataset for extremely low altitude drone flights},
booktitle = {Conference on Computer Vision and Pattern Recognition Workshop (CVPRW)},
year = {2019},
month = {June}
}


OBS: Devido às limitações de espaço no Google Colab, foi necessário realizar diversas rodadas de importação do Dataset, usando o espaço de armazenamento do Google Drive, estas etapas de download, unzip e organização de diretórios foram removidas deste código para melhor visualização.


In [ ]:
# Conexão com o Google Drive para acesso ao Dataset e gravação de resultados 


from google.colab import drive
drive.mount('/content/drive/', force_remount=True)



Mounted at /content/drive/


In [ ]:
# Teste para verificar acesso ao dataset no Colab
!ls -l /content/drive/MyDrive/MIDAIR/train/color_right/

ls: cannot access '/content/drive/MyDrive/MIDAIR/train/color_right/': No such file or directory


In [ ]:
# definição dos parametros de treinamento

dict_parameters = edict({'data_dir':'/content/drive/MyDrive/MIDAIR/train/',
                         'val_data_dir':'/content/drive/MyDrive/MIDAIR/val/',
                         'model_path':'/content/MonoDepth-PyTorch/data/models/monodepth_resnet18_001.pth',
                         'output_directory':'/content/drive/MyDrive/MIDAIR/output/',
                         'input_height':256,
                         'input_width':512,
                         'model':'resnet18_md',
                         'pretrained':True,
                         'mode':'train',
                         'epochs':200, 
                         'learning_rate':1e-4,
                         'batch_size': 2,  #8
                         'adjust_lr':True,
                         'device':'cuda:0',
                         'do_augmentation':True,
                         'augment_parameters':[0.8, 1.2, 0.5, 2.0, 0.8, 1.2],
                         'print_images':False,
                         'print_weights':False,
                         'input_channels': 3,
                         'num_workers': 8,
                         'use_multiple_gpu': False})

In [ ]:
# carga do modelo
model = Model(dict_parameters)
model.load('/content/MonoDepth-PyTorch/data/models/monodepth_resnet18_001.pth')

Use a dataset with 2205 images


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Use a dataset with 2206 images


In [ ]:
# Treinamento do modelo com os parametros especificados
model.train()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3982: UserWarning: Default grid_sample and affine_grid behavior has changed to

Val_loss: 1.4933506832101178
Epoch: 1 train_loss: 1.0793556734473728 val_loss: 1.6680361463369426 time: 407.137 s
Model_saved
Epoch: 2 train_loss: 1.027305135619932 val_loss: 1.6974703065392112 time: 393.748 s
Epoch: 3 train_loss: 1.000401922982061 val_loss: 1.6921991800234701 time: 393.82 s
Epoch: 4 train_loss: 0.9901972546268353 val_loss: 1.7154429065397268 time: 392.33 s
Epoch: 5 train_loss: 0.9651886437118647 val_loss: 1.7000162104089784 time: 391.744 s
Epoch: 6 train_loss: 0.9621686331812944 val_loss: 1.7285687011385722 time: 391.483 s
Epoch: 7 train_loss: 0.9499079134392933 val_loss: 1.7665646745504435 time: 391.355 s
Epoch: 8 train_loss: 0.9491470688219408 val_loss: 1.7532516434079124 time: 394.295 s
Epoch: 9 train_loss: 0.9384023975428947 val_loss: 1.7585994162527072 time: 398.042 s
Epoch: 10 train_loss: 0.9254485166894233 val_loss: 1.7863641995151027 time: 394.693 s
Epoch: 11 train_loss: 0.9235881428527271 val_loss: 1.7760095031893983 time: 394.943 s
Epoch: 12 train_loss: 0.91

## Test the model

In [ ]:
# Verificar o acesso aos dados de teste do modelo

!ls /content/drive/MyDrive/MIDAIR/test

In [ ]:
#Definição dos parametros de teste

dict_parameters_test = edict({'data_dir':'/content/drive/MyDrive/MIDAIR/test',
                              'model_path':'/content/MonoDepth-PyTorch/data/models/monodepth_resnet18_001_cpt.pth',
                              'output_directory':'/content/drive/MyDrive/MIDAIR/output',
                              'input_height':256,
                              'input_width':512,
                              'model':'resnet18_md',
                              'pretrained':False,
                              'mode':'test',
                              'device':'cuda:0',
                              'input_channels':3,
                              'num_workers':4,
                              'use_multiple_gpu':False})
model_test = Model(dict_parameters_test)

In [ ]:
#Execução do teste do modelo
model_test.test()

In [ ]:
# Salvar o NumPy array file (parametros do modelo treinado)

disp = np.load('/content/drive/MyDrive/MIDAIR/output/disparities.npy')  # Or disparities.npy for output without post-processing
disp.shape

In [ ]:
#Exibir imagem de teste em formato "Plasma"
disp_to_img = skimage.transform.resize(disp[0].squeeze(), [375, 1242], mode='constant')
plt.imshow(disp_to_img, cmap='plasma')

Save a color image

In [ ]:
#salvar imagem de teste em formato "Plasma"



plt.imsave(os.path.join(dict_parameters_test.output_directory,
                        dict_parameters_test.model_path.split('/')[-1][:-4]+'_test_output.png'), disp_to_img, cmap='plasma')

Save all test images

In [ ]:
#Salvar todas as imagens geradas no teste

for i in range(disp.shape[0]):
    disp_to_img = skimage.transform.resize(disp[i].squeeze(), [375, 1242], mode='constant')
    plt.imsave(os.path.join(dict_parameters_test.output_directory,
               'pred_'+str(i)+'.png'), disp_to_img, cmap='plasma')

Save a grayscale image

In [ ]:
#Salvar uma imagem de teste como escala de cinzaplt.imsave(os.path.join(dict_parameters_test.output_directory,
                        dict_parameters_test.model_path.split('/')[-1][:-4]+'_gray.png'), disp_to_img, cmap='gray')